In [1]:
# import sys
# import os

# # Calculate the path to the utils module
# current_dir = os.path.dirname(__file__)
# utils_dir = os.path.join(current_dir, "utils")

# # Check if the directory exists
# if os.path.exists(utils_dir) and os.path.isdir(utils_dir):
#     # Add the utils directory to sys.path
#     sys.path.insert(0, utils_dir)
# else:
#     print(f"Could not locate the directory {utils_dir}")

In [2]:
from neuron import h, gui
from neuron.units import ms, mV, µm
nA = 1e-9
import matplotlib.pyplot as plt
import numpy as np

h.load_file("stdrun.hoc")

# enable interactive plotting for jupyter
plt.ion()
%matplotlib notebook

In [3]:
# import utils

In [4]:
def reset_h():
    h.load_file("stdrun.hoc")
    h.stdinit()
    h.dt = 0.025 * ms

In [75]:
class BallAndStick:
    def __init__(self, gid):
        self._gid = gid
        self._setup_morphology()
        self._setup_biophysics()

    def _setup_morphology(self):
        self.soma = h.Section(name="soma", cell=self)
        self.dend = h.Section(name="dend", cell=self)
        self.dend.connect(self.soma)
        self.all = self.soma.wholetree()
        self.soma.L = self.soma.diam = 12.6157 * µm
        self.dend.L = 200 * µm
        self.dend.diam = 1 * µm
        self.dend.nseg = 1

    def _setup_biophysics(self):
        for sec in self.all:
            sec.Ra = 100  # Axial resistance in Ohm * cm
            sec.cm = 1  # Membrane capacitance in micro Farads / cm^2
        self.soma.insert("hh")
        for seg in self.soma:
            seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
            seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
            seg.hh.gl = 0.0003  # Leak conductance in S/cm2
            seg.hh.el = -54.3 * mV  # Reversal potential
        # Insert passive current in the dendrite                       # <-- NEW
        self.dend.insert("pas")  # <-- NEW
        for seg in self.dend:  # <-- NEW
            seg.pas.g = 0.001  # Passive conductance in S/cm2        # <-- NEW
            seg.pas.e = -65 * mV  # Leak reversal potential             # <-- NEW

    def __repr__(self):
        return "BallAndStick[{}]".format(self._gid)

In [76]:
my_cell = BallAndStick(0)
my_other_cell = BallAndStick(1)

In [77]:
for sec in h.allsec():
    print("%s: %s" % (sec, ", ".join(sec.psection()["density_mechs"].keys())))

BallAndStick[1].dend: pas
BallAndStick[0].soma: hh
BallAndStick[0].dend: pas
BallAndStick[1].soma: hh
BallAndStick[1].dend: pas


In [78]:
# stim = h.IClamp(my_cell.dend(1))


class Clamp:
    def __init__(self, stim, delay, dur, amp):
        self.stim = stim
        self.stim.delay = delay
        self.stim.dur = dur
        self.stim.amp = amp

    def __repr__(self):
        return "Clamp[{}]".format(self.stim)


clamp = Clamp(h.IClamp(my_cell.dend(1)), delay=5, dur=1, amp=0.1)

In [79]:
def setup_h():
    reset_h()

    my_cell = BallAndStick(0)
    my_other_cell = BallAndStick(1)

    clamp = Clamp(h.IClamp(my_cell.dend(1)), delay=5 * ms, dur=1 * ms, amp=0.1)

    return clamp

In [80]:
# utils.get_attributes(clamp.stim)

In [81]:
h.finitialize(-65 * mV)
h.continuerun(25 * ms)

0.0

In [82]:
from bokeh.io import output_notebook
import bokeh.plotting as plt

output_notebook()

Loading BokehJS ...

In [83]:
reset_h()

my_cell = BallAndStick(0)
my_other_cell = BallAndStick(1)

clamp = Clamp(h.IClamp(my_cell.dend(1)), delay=5 * ms, dur=1 * ms, amp=0.1)

soma_v = h.Vector().record(my_cell.soma(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

h.finitialize(-65 * mV)
h.continuerun(25 * ms)

_soma_v = np.array(soma_v)
_t = np.array(t)

f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)")
f.line(_t, _soma_v, line_width=2)
plt.show(f)

In [84]:
reset_h()
my_cell = BallAndStick(0)
my_other_cell = BallAndStick(1)
clamp = Clamp(h.IClamp(my_cell.dend(1)), delay=5, dur=1, amp=0.1)

soma_v = h.Vector().record(my_cell.soma(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

In [85]:
h.topology()


|-|       BallAndStick[1].dend(0-1)
|-|       BallAndStick[0].soma(0-1)
   `|       BallAndStick[0].dend(0-1)
|-|       BallAndStick[1].soma(0-1)
   `|       BallAndStick[1].dend(0-1)



1.0

In [86]:
ps = h.PlotShape(True)
ps.show(0)

1.0

In [87]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)")
amps = [0.75 * i for i in range(1, 5)]
colors = ["green", "blue", "red", "black"]
for amp, color in zip(amps, colors):
    clamp.stim.amp = amp

    h.finitialize(-65 * mV)
    h.continuerun(25 * ms)

    _soma_v = np.array(soma_v)
    _t = np.array(t)

    f.line(_t, list(_soma_v), line_width=2, legend_label="amp=%g" % amp, color=color)
plt.show(f)

In [97]:
reset_h()
my_cell = BallAndStick(0)
my_other_cell = BallAndStick(1)
clamp = Clamp(h.IClamp(my_cell.dend(1)), delay=5, dur=1, amp=0.1)

# for sec in h.allsec():
#     length_constant = 1e5*np.sqrt(sec.diam/(4*np.pi*100*sec.Ra*sec.cm))
#     sec.nseg = int((sec.L/(length_constant)+0.9)/2)*2+1
#     print(sec.nseg)

soma_v = h.Vector().record(my_cell.soma(0.5)._ref_v)
dend_v = h.Vector().record(my_cell.dend(0.1)._ref_v)
dend_v_2 = h.Vector().record(my_cell.dend(1)._ref_v)
t = h.Vector().record(h._ref_t)

In [98]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)")
amps = [0.5]
colors = ["green", "blue", "red", "black"]
for amp, color in zip(amps, colors):
    clamp.stim.amp = amp

    h.finitialize(-65 * mV)
    h.continuerun(25 * ms)

    _soma_v = np.array(soma_v)
    _dend_v = np.array(dend_v)
    _dend_v_2 = np.array(dend_v_2)
    _t = np.array(t)

    f.line(
        _t,
        list(_soma_v),
        line_width=2,
        legend_label="amp=%g" % amp,
        color=color,
        alpha=0.5,
    )
    # f.line(_t, list(_dend_v), line_width=2, line_dash="dashed", color=color, alpha=0.5)
    f.line(
        _t, list(_dend_v_2), line_width=2, line_dash="dotted", color=color, alpha=0.5
    )
    # bokeg set y lim
    f.y_range.start = -80
    f.y_range.end = 50


plt.show(f)

In [99]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)")
for amp, color in zip(amps, colors):
    clamp.stim.amp = amp

    h.finitialize(-65 * mV)
    h.continuerun(25 * ms)

    _soma_v = np.array(soma_v)
    _dend_v = np.array(dend_v)
    _dend_v_2 = np.array(dend_v_2)
    _t = np.array(t)

    f.line(
        _t,
        list(_soma_v),
        line_width=2,
        legend_label="amp=%g" % amp,
        color=color,
        alpha=0.5,
    )
    f.line(_t, list(_dend_v), line_width=2, line_dash="dashed", color=color, alpha=0.5)
    # f.line(_t, list(_dend_v_2), line_width=2, line_dash="dotted", color=color, alpha=0.5)
    f.y_range.start = -80
    f.y_range.end = 50

plt.show(f)